In [40]:
import pandas as pd 
import os 
import numpy as np 
import matplotlib.pyplot as plt 

import matplotlib
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica Neue') 
matplotlib.rc('text', usetex='false') 

from GL_utils import data_loading as GL_load
import importlib 
importlib.reload(GL_load)

csv_dir = "csv"
working_dir = "/Users/evanlee/MSTP/Gordon/experiments/MG00_HWDC_og/HOSVD"
os.chdir(working_dir)

DE_dir = os.path.join(working_dir,"DE")
DE_input_dir = os.path.join(DE_dir,"input/edgeR_filt")
DE_output_dir = os.path.join(DE_dir,"output/DE_LRT_edgeR_filt_E6nf")
DE_supplementary_dir = os.path.join(DE_dir,"supplementary_tables/DE_mcseed_cazyme")
os.makedirs(DE_supplementary_dir,exist_ok=True)

In [8]:
#Deprecated - SVD average pathway expression for supplemental table 
DATASET = "rlog"
SVD_parent_dir = "/Users/evanlee/MSTP/Gordon/experiments/MG00_HWDC_og/HOSVD"
bpm_filt_data_path = "{0}/formatted_output/4th_trial/edgeR_filtered".format(SVD_parent_dir) #parent directory for SVD dataset 
edgeR_input_path = "{0}/DE/input/edgeR_filt".format(SVD_parent_dir)
cecal_LRT_path = "{0}/DE/output/DE_LRT_edgeR_filt_E6nf".format(SVD_parent_dir)
assert(os.path.exists(bpm_filt_data_path))
assert(os.path.exists(edgeR_input_path))
assert(os.path.exists(cecal_LRT_path))
condensed_agg_expr_fpath = "{0}/condensed_agg_pht_{1}.csv".format(bpm_filt_data_path,DATASET) #bpm aggregated SVD dataset 
mcSEED_fpath = "{0}/19isolates_mcseed_pathwaycomplete.csv".format(edgeR_input_path)
assert(os.path.exists(condensed_agg_expr_fpath))
assert(os.path.exists(mcSEED_fpath))

In [22]:
#mcSEED and CAZyme loading 


mcseed_fname = "19isolates_mcseed_pathwaycomplete.csv"
mcseed_fpath = os.path.join(DE_input_dir,mcseed_fname)
mcseed_df, pht_pathway_df = GL_load.load_mcSEED(mcseed_fpath,list_cols=False)

cazyme_fname = "CAZyme_annotation_summary.txt"
cazyme_fpath = os.path.join(DE_input_dir,cazyme_fname)
cazyme = GL_load.load_CAZyme(cazyme_fpath)
cazyme.loc[:,"Strain"] = cazyme.loc[:,"Strain"].replace(to_replace="Bwe",value="Lru")

display(mcseed_df)
display(cazyme)
display(cazyme["Strain"].unique())

,Isolate name,Protein name,Protein product,Functional category,Functional pathway,Phenotype,Strain
Locus tag,,,,,,,
ANCJAENF_00011,Bifidobacterium breve Bgsng463_m5_93,MalE,"Maltose/maltodextrin ABC transporter, substrat...",Carbohydrate utilization,maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,Bbr
ANCJAENF_00013,Bifidobacterium breve Bgsng463_m5_93,MalF,"Maltose/maltodextrin ABC transporter, permease...",Carbohydrate utilization,maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,Bbr
ANCJAENF_00014,Bifidobacterium breve Bgsng463_m5_93,MalG,"Maltose/maltodextrin ABC transporter, permease...",Carbohydrate utilization,maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,Bbr
ANCJAENF_00052,Bifidobacterium breve Bgsng463_m5_93,GalE,UDP-glucose 4-epimerase (EC 5.1.3.2),Carbohydrate utilization,galactose utilization; lactose utilization,Gal; Lac,Bbr
ANCJAENF_00063,Bifidobacterium breve Bgsng463_m5_93,GalE,UDP-glucose 4-epimerase (EC 5.1.3.2),Carbohydrate utilization,galactose utilization; lactose utilization,Gal; Lac,Bbr
...,...,...,...,...,...,...,...
LDOIJNDB_02225,Streptococcus pasteriuanus Bang_SAM2_39_S1,TreB_c,"PTS system, trehalose-specific IIC component (...",Carbohydrate utilization,trehalose utilization,Tre,Spa
LDOIJNDB_02226,Streptococcus pasteriuanus Bang_SAM2_39_S1,TreB_b,"PTS system, trehalose-specific IIB component (...",Carbohydrate utilization,trehalose utilization,Tre,Spa
LDOIJNDB_02245,Streptococcus pasteriuanus Bang_SAM2_39_S1,GalE,UDP-glucose 4-epimerase (EC 5.1.3.2),Carbohydrate utilization,galactose utilization; lactose utilization,Gal; Lac,Spa


,Isolate,Locus tag,GH,Model notes,Defline,Strain
0,bifidobacterium_breve_bgsng463_m5_93,ANCJAENF_00037,GH43,fragment N-term;,Glycoside Hydrolase Family 43 protein,Bbr
1,bifidobacterium_breve_bgsng463_m5_93,ANCJAENF_00038,GH43_22,fragment N-term / C-term;,Glycoside Hydrolase Family 43 protein,Bbr
2,bifidobacterium_breve_bgsng463_m5_93,ANCJAENF_00049,GH25,NaN,Glycoside Hydrolase Family 25 protein,Bbr
3,bifidobacterium_breve_bgsng463_m5_93,ANCJAENF_00055,GT2,NaN,Glycosyltransferase Family 2 protein,Bbr
4,bifidobacterium_breve_bgsng463_m5_93,ANCJAENF_00073,GT2,NaN,Glycosyltransferase Family 2 protein,Bbr
...,...,...,...,...,...,...
1748,streptococcus_pasteurianus_bang_sam2_39,LDOIJNDB_02115,GH38,NaN,Glycoside Hydrolase Family 38 protein,Spa
1749,streptococcus_pasteurianus_bang_sam2_39,LDOIJNDB_02123,GT2,NaN,Glycosyltransferase Family 2 protein,Spa
1750,streptococcus_pasteurianus_bang_sam2_39,LDOIJNDB_02125,GT4,NaN,Glycosyltransferase Family 4 protein,Spa
1751,streptococcus_pasteurianus_bang_sam2_39,LDOIJNDB_02126,GT2,NaN,Glycosyltransferase Family 2 protein,Spa


array(['Bbr', 'Bca', 'Bli2D9', 'Bli463', 'Blu', 'Lru', 'Dfo', 'Dlo',
       'Eav', 'Eco', 'FprB', 'Lga4B6', 'Mmu', 'Pco', 'Pst', 'Rgn', 'Rob',
       'Rto', 'Sga', 'Spa'], dtype=object)

In [58]:
display(cazyme[cazyme["Strain"]=="Rob"])
# display(mcseed_df[mcseed_df["Strain"]=="Lru"])

,Isolate,Locus tag,GH,Model notes,Defline,Strain
1414,ruminococcus_obeum_bg7063,FEGMOGGF_00051,GH2,NaN,Glycoside Hydrolase Family 2 protein,Rob
1415,ruminococcus_obeum_bg7063,FEGMOGGF_00052,GH53-CBM13,NaN,Glycoside Hydrolase Family 53 / Carbohydrate-...,Rob
1416,ruminococcus_obeum_bg7063,FEGMOGGF_00090,GT4,fragment N-term;,Glycosyltransferase Family 4 protein,Rob
1417,ruminococcus_obeum_bg7063,FEGMOGGF_00103,GH1,NaN,Glycoside Hydrolase Family 1 protein,Rob
1418,ruminococcus_obeum_bg7063,FEGMOGGF_00127,CBM48-GH13_9,NaN,Carbohydrate-Binding Module Family 48 / Glyco...,Rob
...,...,...,...,...,...,...
1504,ruminococcus_obeum_bg7063,FEGMOGGF_02843,GH3,NaN,Glycoside Hydrolase Family 3 protein,Rob
1505,ruminococcus_obeum_bg7063,FEGMOGGF_02889,GH1,NaN,Glycoside Hydrolase Family 1 protein,Rob
1506,ruminococcus_obeum_bg7063,FEGMOGGF_02893,GH106,NaN,Glycoside Hydrolase Family 106 protein,Rob
1507,ruminococcus_obeum_bg7063,FEGMOGGF_02894,GH1,NaN,Glycoside Hydrolase Family 1 protein,Rob


In [57]:
# strain = "Bbr"
DE_strains = ["Bbr","Bca","Dlo","Eav","Eco","Lga4B6","Lru","Mmu","Rgn","Rob","Rto","Sga","Spa"]

for strain in DE_strains:
    #Load all DE data for strain
    strain_all_fname = "{0}_cecal_p1Cn2B_all.csv".format(strain)
    strain_all_fpath = os.path.join(DE_output_dir,strain_all_fname)
    assert(os.path.exists(strain_all_fpath))
    strain_all_DE = pd.read_csv(strain_all_fpath,index_col=0)
    strain_all_DE.index.name = "Locus tag"

    #Strain filtered cazyme and mcseed - requires previous fixing of strain abbreviations if discrepancy between CAZyme 
    #table and DE tables (i.e. Bwe used for Lru)
    strain_cazyme = cazyme.loc[cazyme["Strain"]==strain].set_index("Locus tag")
    strain_mcseed = mcseed_df.loc[mcseed_df["Strain"]==strain]
    #Strain filtered significant DE, then strain mcSEED or CAZyme annotated significant DE 
    strain_sig_DE = strain_all_DE.loc[strain_all_DE["FDR"]<=0.05]
    strain_annotated_sig_DE = strain_sig_DE.loc[(strain_sig_DE.index.isin(strain_cazyme.index)) |
                                               (strain_sig_DE.index.isin(strain_mcseed.index))]
    #Filter strain sig cazyme/mcSEED down to significantly DE genes 
    strain_sig_cazyme = strain_cazyme.loc[strain_cazyme.index.isin(strain_annotated_sig_DE.index),:]
    strain_sig_mcseed = strain_mcseed.loc[strain_mcseed.index.isin(strain_annotated_sig_DE.index),:]
    #Supplementary table composition - first populate cazyme and DE information (since no duplicate locus entries),
    #then join in mcSEED information (which has duplicate locus entries for different protein products), reorder to 
    #final Supplementary Table format/order 
    supplementary_table_mcseed_columns = ["Protein name","Protein product","Functional pathway","Phenotype"]
    supplementary_table_cazyme_columns = ["GH","Defline"]
    supplementary_table_DE_columns = ["logFC","FDR"]

    supplementary_cazyme_DE_cols = np.concatenate([supplementary_table_cazyme_columns,supplementary_table_DE_columns])
    supplementary_table_cols = np.concatenate([supplementary_table_mcseed_columns,supplementary_table_cazyme_columns,
                                               supplementary_table_DE_columns])
    #Populate CAZyme and DE information 
    strain_supplementary_table = pd.DataFrame(index=strain_annotated_sig_DE.index,columns=supplementary_cazyme_DE_cols)
    strain_supplementary_table.loc[:,supplementary_table_DE_columns] = strain_annotated_sig_DE.loc[:,supplementary_table_DE_columns]
    strain_supplementary_table.loc[:,supplementary_table_cazyme_columns] = strain_sig_cazyme.loc[:,supplementary_table_cazyme_columns]
    #Join in mcSEED information to produce appropriate duplicate rows for different mcSEED protein products 
    strain_sig_mcseed_supplementary = strain_sig_mcseed.loc[:,supplementary_table_mcseed_columns]
    strain_supplementary_table = strain_supplementary_table.join(strain_sig_mcseed_supplementary,how="left")
    strain_supplementary_table = strain_supplementary_table.loc[:,supplementary_table_cols]
    display(strain_supplementary_table)

    strain_supplementary_table_fpath = os.path.join(DE_supplementary_dir,"{0}_sig_mcSEED_CAZyme.csv".format(strain))
    strain_supplementary_table.to_csv(strain_supplementary_table_fpath)

,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
ANCJAENF_00419,NaN,NaN,NaN,NaN,GT2,Glycosyltransferase Family 2 protein,-1.71754,0.036781
ANCJAENF_00928,TrpC,Indole-3-glycerol phosphate synthase (EC 4.1.1...,tryptophan biosynthesis,Trp,NaN,NaN,-6.302285,0.012585
ANCJAENF_00928,TrpB,Tryptophan synthase beta chain (EC 4.2.1.20),tryptophan biosynthesis,Trp,NaN,NaN,-6.302285,0.012585


,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
NNMNEBCK_00010,AbfB,Double substituted xylan alpha-(1->3)-L-arabin...,alpha-arabinooligosaccharides utilization,aAOS,GH51,Glycoside Hydrolase Family 51 protein,-1.478481,0.000785
NNMNEBCK_00011,NaN,NaN,NaN,NaN,GH27,Glycoside Hydrolase Family 27 protein,-1.436704,0.002078
NNMNEBCK_00013,NaN,NaN,NaN,NaN,GH36,Glycoside Hydrolase Family 36 protein,1.76603,0.003123
NNMNEBCK_00025,NaN,NaN,NaN,NaN,CE9,Carbohydrate Esterase Family 9 protein,1.746492,0.005451
NNMNEBCK_00027,NaN,NaN,NaN,NaN,GH20,Glycoside Hydrolase Family 20 protein,1.713329,0.039905
...,...,...,...,...,...,...,...,...
NNMNEBCK_01770,GosA,b(1-4)-linked galactooligosaccharides ABC tran...,galactooligosaccharides utilization,GOS,NaN,NaN,3.646362,0.000047
NNMNEBCK_01778,AraQ_LacI,"AraQ transcriptional regulator, LacI family",arabinose utilization,Ara,NaN,NaN,-1.496449,0.010629
NNMNEBCK_01824,GatC,Aspartyl-tRNA(Asn) amidotransferase subunit C ...,asparagine biosynthesis; glutamine biosynthesis,Asn; Gln,NaN,NaN,-1.109643,0.04688


,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
OOAPABDJ_02350,NanK,N-acetylmannosamine kinase (EC 2.7.1.60),N-acetylmannosamine utilization; N-acetylneura...,ManNAc; NANA,NaN,NaN,4.174821,0.000022
OOAPABDJ_02349,NanR7,"Sialic acid utilization regulator, RpiR family",N-acetylneuraminate utilization,NANA,NaN,NaN,4.495938,0.000045
OOAPABDJ_02357,NanR4,Transcriptional regulator of silaic acid utili...,N-acetylneuraminate utilization,NANA,NaN,NaN,3.271127,0.000838
OOAPABDJ_02354,NanX2,"ABC transporter, predicted N-acetylneuraminate...",N-acetylneuraminate utilization,NANA,NaN,NaN,5.397672,0.001087
OOAPABDJ_02352,NanH,Sialidase (EC 3.2.1.18),N-acetylneuraminate utilization,NANA,CBM40-GH33,Carbohydrate-Binding Module Family 40 / Glyco...,5.701513,0.002022
OOAPABDJ_02355,NanX1,"ABC transporter, predicted N-acetylneuraminate...",N-acetylneuraminate utilization,NANA,NaN,NaN,5.680141,0.002071
OOAPABDJ_02351,NanA,N-acetylneuraminate lyase (EC 4.1.3.3),N-acetylneuraminate utilization,NANA,NaN,NaN,5.308406,0.002502
OOAPABDJ_02356,NanY,"ABC transporter, predicted N-acetylneuraminate...",N-acetylneuraminate utilization,NANA,NaN,NaN,5.786958,0.005907
OOAPABDJ_02710,MalF,"Maltose/maltodextrin ABC transporter, permease...",maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,NaN,NaN,-6.145429,0.010176


,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
AKLJOBCP_00364,PflA,Pyruvate formate-lyase (EC 2.3.1.54),formate production,Formate,NaN,NaN,-0.899211,0.014161
AKLJOBCP_00368,CelA,"PTS system, cellobiose-specific IIA component ...",beta-glucosides utilization,Bgl,NaN,NaN,1.618399,0.028687
AKLJOBCP_00369,CelC,"PTS system, cellobiose-specific IIC component ...",beta-glucosides utilization,Bgl,NaN,NaN,1.699806,0.025372
AKLJOBCP_00370,CelB1,"PTS system, cellobiose-specific IIB component ...",beta-glucosides utilization,Bgl,NaN,NaN,2.37728,0.000268
AKLJOBCP_00385,CelC,"PTS system, cellobiose-specific IIC component ...",beta-glucosides utilization,Bgl,NaN,NaN,1.207943,0.000036
...,...,...,...,...,...,...,...,...
AKLJOBCP_04327,GatPc,"PTS system, galactitol-specific IIC component ...",galactitol utilization,Gtl,NaN,NaN,2.441361,0.0
AKLJOBCP_04347,GlvA,"PTS system, maltose and glucose-specific IIA c...",glucose utilization; maltose utilization; malt...,Glc; Mal; (Mal)n,NaN,NaN,-0.959508,0.018627
AKLJOBCP_04350,GlvG,Maltose-6'-phosphate glucosidase (EC 3.2.1.122),maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,GH4,Glycoside Hydrolase Family 4 protein,-1.422878,0.004912


,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
HMLHAAEP_00054,PtsGb,"PTS system, glucose-specific IIB component (EC...",glucose utilization,Glc,NaN,NaN,-0.838404,0.047833
HMLHAAEP_00054,PtsGc,"PTS system, glucose-specific IIC component (EC...",glucose utilization,Glc,NaN,NaN,-0.838404,0.047833
HMLHAAEP_00251,LysA,Diaminopimelate decarboxylase (EC 4.1.1.20),lysine biosynthesis,Lys,NaN,NaN,1.437871,0.000189
HMLHAAEP_00255,KduD,2-deoxy-D-gluconate 3-dehydrogenase (EC 1.1.1....,unsaturated glucuronate utilization,ddGlcA,NaN,NaN,1.564391,0.000079
HMLHAAEP_00256,KduI,4-deoxy-L-threo-5-hexosulose-uronate ketol-iso...,unsaturated glucuronate utilization,ddGlcA,NaN,NaN,1.930911,0.000023
...,...,...,...,...,...,...,...,...
HMLHAAEP_04667,EutP,Ethanolamine utilization protein EutP,ethanolamine utilization,EA_ut,NaN,NaN,1.13625,0.021278
HMLHAAEP_04944,AspC,Aspartate aminotransferase (EC 2.6.1.1),aspartate biosynthesis,Asp,NaN,NaN,-1.605625,0.003217
HMLHAAEP_04966,DapA,4-hydroxy-tetrahydrodipicolinate synthase (EC ...,lysine biosynthesis,Lys,NaN,NaN,1.690418,0.000745


...

,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
GJBELKAJ_00091,BglA,Beta-glucosidase (EC 3.2.1.21),beta-glucosides utilization,Bgl,GH3,Glycoside Hydrolase Family 3 protein,-1.686097,0.00361
GJBELKAJ_00092,IolG1,Myo-inositol 2-dehydrogenase 1 (EC 1.1.1.18),inositol utilization,Ino,NaN,NaN,-1.695457,0.026924
GJBELKAJ_00093,IolI,Inosose isomerase (EC 5.3.99.11),inositol utilization,Ino,NaN,NaN,-1.661416,0.007417
GJBELKAJ_00096,InoF,"Predicted inositol ABC transporter, substrate-...",inositol utilization,Ino,NaN,NaN,-1.756065,0.00952
GJBELKAJ_00097,InoG,"Predicted inositol ABC transporter, permease p...",inositol utilization,Ino,NaN,NaN,-1.519862,0.02033
...,...,...,...,...,...,...,...,...
GJBELKAJ_03734,PulA,Pullulanase (EC 3.2.1.41),maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,CBM48-GH13_14,Carbohydrate-Binding Module Family 48 / Glyco...,-2.159392,0.0
GJBELKAJ_03735,MalG,"Maltose/maltodextrin ABC transporter, permease...",maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,NaN,NaN,-2.272436,0.0
GJBELKAJ_03736,MalF,"Maltose/maltodextrin ABC transporter, permease...",maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,NaN,NaN,-2.213113,0.0


,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
HIAFFLEM_00101,NadD,Nicotinate-nucleotide adenylyltransferase (EC ...,"NAD(P) cofactors, de novo synthesis",B3,NaN,NaN,8.951983,0.0
HIAFFLEM_00109,AroB,3-dehydroquinate synthase (EC 4.2.3.4),chorismate biosynthesis,Chor,NaN,NaN,8.371403,0.0
HIAFFLEM_00136,HisI,Phosphoribosyl-AMP cyclohydrolase (EC 3.5.4.19),histidine biosynthesis,His,NaN,NaN,9.770764,0.0
HIAFFLEM_00136,HisE,Phosphoribosyl-ATP pyrophosphatase (EC 3.6.1.31),histidine biosynthesis,His,NaN,NaN,9.770764,0.0
HIAFFLEM_00137,HisB,Imidazoleglycerol-phosphate dehydratase (EC 4....,histidine biosynthesis,His,NaN,NaN,8.345493,0.0
...,...,...,...,...,...,...,...,...
HIAFFLEM_03409,ProA,Gamma-glutamyl phosphate reductase (EC 1.2.1.41),proline biosynthesis,Pro,NaN,NaN,10.595455,0.0
HIAFFLEM_03427,AlfA,Alpha-L-fucosidase (EC 3.2.1.51),fucosides utilization,Fcs,NaN,NaN,7.57569,0.000007
HIAFFLEM_03431,FucX,"Fucose ABC transport system, substrate-binding...",fucose utilization,Fuc,NaN,NaN,6.824848,0.001961


,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
CGCJBKJN_00427,NaN,NaN,NaN,NaN,GT2,Glycosyltransferase Family 2 protein,6.171981,0.0
CGCJBKJN_00424,NaN,NaN,NaN,NaN,GT4,Glycosyltransferase Family 4 protein,4.461258,0.0
CGCJBKJN_00429,NaN,NaN,NaN,NaN,GT2,Glycosyltransferase Family 2 protein,4.094038,0.0
CGCJBKJN_00195,RhaA,L-rhamnose isomerase (EC 5.3.1.14),rhamnose utilization; rhamnogalacturonides uti...,Rha; Rhi,NaN,NaN,2.278743,0.000009
CGCJBKJN_01756,BglX,"Beta-glucosides ABC transporter, substrate-bin...",beta-glucosides utilization,Bgl,NaN,NaN,-3.556347,0.000697
CGCJBKJN_02069,BfrR,"Transcriptional regulator of FOS utilization, ...",fructooligosaccharides utilization,FOS,NaN,NaN,2.588885,0.001056
CGCJBKJN_02737,MalQ,4-alpha-glucanotransferase (amylomaltase) (EC ...,maltose utilization; maltooligosaccharides uti...,Mal; (Mal)n,GH77,Glycoside Hydrolase Family 77 protein,2.250942,0.002847
CGCJBKJN_00402,MetE2,Methionine synthase II (cobalamin-independent),methionine biosynthesis,Met,NaN,NaN,2.438657,0.00306
CGCJBKJN_00009,NagR3,Predicted antiterminator of N-acetylglucosamin...,N-acetylglucosamine utilization,GlcNAc,NaN,NaN,2.628793,0.005965


,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
JKAMACKM_01660,Frk_ScrK; ScrK,Fructokinase (EC 2.7.1.4),fructose utilization; sucrose utilization; fru...,Fru; Scr; FOS; Mtl,NaN,NaN,-1.964363,0.043397


,Protein name,Protein product,Functional pathway,Phenotype,GH,Defline,logFC,FDR
Locus tag,,,,,,,,
LDOIJNDB_00083,ArtP2,"Arginine ABC transporter, ATP-binding protein",arginine biosynthesis,Arg,NaN,NaN,-3.69435,0.0
LDOIJNDB_00220,GlcK,Glucokinase (EC 2.7.1.2),glucose utilization,Glc,NaN,NaN,-1.160114,0.042402
LDOIJNDB_00473,ScrAa1,"PTS system, sucrose-specific IIA component (EC...",sucrose utilization,Scr,NaN,NaN,1.43312,0.006201
LDOIJNDB_00473,ScrAb1,"PTS system, sucrose-specific IIB component (EC...",sucrose utilization,Scr,NaN,NaN,1.43312,0.006201
LDOIJNDB_00474,NaN,NaN,NaN,NaN,GH13_31,Glycoside Hydrolase Family 13 protein,1.353175,0.017711
...,...,...,...,...,...,...,...,...
LDOIJNDB_02033,LacEb,"PTS system, lactose-specific IIB component (EC...",lactose utilization,Lac,NaN,NaN,-1.088014,0.023604
LDOIJNDB_02033,LacEc,"PTS system, lactose-specific IIC component (EC...",lactose utilization,Lac,NaN,NaN,-1.088014,0.023604
LDOIJNDB_02070,NaN,NaN,NaN,NaN,GT8,Glycosyltransferase Family 8 protein,1.027776,0.021896
